In [7]:
# Import required libraries
import pandas as pd

# Load the Excel file
file_path = '/content/drive/MyDrive/BID/ULTIMOS/bolivia/input/datos_bolivia_compilados.xlsx'
xls = pd.ExcelFile(file_path)

# Load the data from both sheets
ha_df = pd.read_excel(xls, sheet_name='ha')
ton_df = pd.read_excel(xls, sheet_name='ton')

# Reshaping the data to the desired format using pandas' melt function

# Transform the "ha" dataframe
ha_df_melted = ha_df.melt(id_vars=['Departamento', 'Cultivo'],
                          var_name='Año',
                          value_name='ha')

# Transform the "ton" dataframe
ton_df_melted = ton_df.melt(id_vars=['Departamento', 'Cultivo'],
                            var_name='Año',
                            value_name='ton')

# Replace non-numeric values in the 'ha' and 'ton' columns with 0

# For ha_df_melted
ha_df_melted['ha'] = pd.to_numeric(ha_df_melted['ha'], errors='coerce').fillna(0)

# For ton_df_melted
ton_df_melted['ton'] = pd.to_numeric(ton_df_melted['ton'], errors='coerce').fillna(0)

# Define the translation dictionary
translations = {
    "Palma Africana (Fruta Fresca)": "Oil palm",
    "Maíz": "Corn",
    "Arroz con cáscara": "Rice",
    "Soya": "Soy",
    "Plátano (Fruta Fresca)": "Plantain",
    'Caña de Azúcar': 'Sugarcane',
    'Caña De Azúcar Para Otros Usos (Tallo Fresco)': 'Sugarcane',
    "Yuca (Raíz Fresca)": "Yuca",
    "Cacao (Almendra Seca)": "Cocoa",
    "MARAÑÓN": "Cashew",
    "Café (Grano Oro)": "Coffee",
    'Sorgo en grano': 'Sorghum',
    'Trigo': 'Wheat',
    'Girasol': 'Sunflower',
    'Papa': 'Potato',
    'Frijol': 'Bean',
    'Alfalfa': 'Alfalfa',
    'Cebada': 'Barley',
    'Maíz Duro Seco (Grano Seco)': 'Corn',
    'Maíz Duro Choclo (En Choclo)': 'Corn',
    'Maíz Suave Choclo (En Choclo)': 'Corn',
    'Maíz Suave Seco (Grano Seco)': 'Corn',
    'Frijol/poroto': 'Bean',
    'Fréjol Tierno (En Vaina)': 'Bean',
    'Naranja (Fruta Fresca)': 'Orange',
    'Orito (Fruta Fresca)': 'Orito',
    'Maíz en grano': 'Corn',
    'Alfalfa': 'Alfalfa'
}

# Apply the translation to the 'Cultivo' column in both datasets
ha_df_melted['Cultivo'] = ha_df_melted['Cultivo'].replace(translations)
ton_df_melted['Cultivo'] = ton_df_melted['Cultivo'].replace(translations)

# Filter data for the specific departments
departments_to_include = ["Beni", "Cochabamba", "Chuquisaca", "La Paz", "Pando", "Santa Cruz"]
ha_filtered = ha_df_melted[ha_df_melted['Departamento'].isin(departments_to_include)]
ton_filtered = ton_df_melted[ton_df_melted['Departamento'].isin(departments_to_include)]

# Filter the data for the year 2023
ha_2023 = ha_filtered[ha_filtered['Año'] == 2023]
ton_2023 = ton_filtered[ton_filtered['Año'] == 2023]

# Identify the top 10 crops by total hectares in 2023
top_10_crops_ha = ha_2023.groupby('Cultivo')['ha'].sum().nlargest(10).index

# Filter the datasets to include only the top 10 crops
ha_top_10 = ha_2023[ha_2023['Cultivo'].isin(top_10_crops_ha)]
ton_top_10 = ton_2023[ton_2023['Cultivo'].isin(top_10_crops_ha)]

# Merge the two datasets on Departamento, Cultivo, and Año to get both ha and ton for 2023
merged_top_10_2023 = pd.merge(ha_top_10, ton_top_10, on=['Departamento', 'Cultivo', 'Año'], suffixes=('_ha', '_ton'))

# Group by 'Cultivo' and calculate the total hectares and tons for each crop
summary_top_10_2023 = merged_top_10_2023.groupby('Cultivo').agg({'ha': 'sum', 'ton': 'sum'}).reset_index()

# Prepare the dataframe in the desired structure for Departments
# Create a list of unique departments for 2023 filtered data
departments_2023 = merged_top_10_2023['Departamento'].unique()
structured_data = []

# Iterate over each department
for dept in departments_2023:
    # Filter data for the current department
    dept_data = merged_top_10_2023[merged_top_10_2023['Departamento'] == dept]

    # Create a dictionary for hectares (ha) row
    ha_row = {'Departamento': dept, 'Indicador': 'ha'}
    # Create a dictionary for tonnes (ton) row
    ton_row = {'Departamento': dept, 'Indicador': 'ton'}

    # Populate the hectares and tonnes values for each crop in the top 10 list
    for _, row in dept_data.iterrows():
        ha_row[row['Cultivo']] = row['ha']
        ton_row[row['Cultivo']] = row['ton']

    # Append the rows to the list
    structured_data.append(ha_row)
    structured_data.append(ton_row)

# Create a dataframe from the list of dictionaries
structured_df = pd.DataFrame(structured_data)

# Prepare the dataframe in the desired structure based on Cultivo and Indicator, with columns for each Department
# Create a list of unique crops in the top 10 list for 2023
crops_2023 = merged_top_10_2023['Cultivo'].unique()
structured_data_cultivo = []

# Iterate over each crop
for crop in crops_2023:
    # Filter data for the current crop
    crop_data = merged_top_10_2023[merged_top_10_2023['Cultivo'] == crop]

    # Create a dictionary for hectares (ha) row
    ha_row = {'Cultivo': crop, 'Indicador': 'ha'}
    # Create a dictionary for tonnes (ton) row
    ton_row = {'Cultivo': crop, 'Indicador': 'ton'}

    # Populate the hectares and tonnes values for each department in the dataset
    for _, row in crop_data.iterrows():
        ha_row[row['Departamento']] = row['ha']
        ton_row[row['Departamento']] = row['ton']

    # Append the rows to the list
    structured_data_cultivo.append(ha_row)
    structured_data_cultivo.append(ton_row)

# Create a dataframe from the list of dictionaries
structured_df_cultivo = pd.DataFrame(structured_data_cultivo)

# Define the path to save the new Excel file
output_file_path = '/content/drive/MyDrive/BID/ULTIMOS/bolivia/output/tablas_cultivo_bolivia.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Write each dataframe to a different worksheet
    summary_top_10_2023.to_excel(writer, sheet_name='Resumen_Top_10', index=False)
    structured_df.to_excel(writer, sheet_name='Deptos_Ha_Ton', index=False)
    structured_df_cultivo.to_excel(writer, sheet_name='Cultivos_Ha_Ton', index=False)

# Provide the path to the user for downloading the file
output_file_path


'/content/drive/MyDrive/BID/ULTIMOS/bolivia/output/tablas_cultivo_bolivia.xlsx'

In [6]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.3 MB/s eta 0:00:00
